# W13 — Zaawansowane biblioteki: scikit-learn, Plotly, Polars

**Programowanie w Pythonie II** | Politechnika Opolska  
**Temat:** Pierwsze modele ML, interaktywne wykresy, szybsze DataFrames

---

## Plan wykładu
1. **scikit-learn** — KMeans clustering (segmentacja klientów)
2. **scikit-learn** — Regresja liniowa (prognoza sprzedaży)
3. **Plotly Express** — interaktywne wykresy (scatter, bar, line)
4. **Polars** — brief mention, benchmark vs Pandas

In [ ]:
# Komórka 1 — Instalacja brakujących bibliotek
# Uruchom raz na początku zajęć
%pip install scikit-learn plotly -q

In [ ]:
# Komórka 2 — Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import sklearn
import plotly
print(f"scikit-learn: {sklearn.__version__}")
print(f"plotly: {plotly.__version__}")
print("Wszystkie importy OK!")

---
## CZĘŚĆ 1: scikit-learn — KMeans Clustering

### Scenariusz biznesowy
Jesteś analitykiem w firmie e-commerce. Masz 300 klientów z dwoma cechami:
- **Średnia wartość zamówienia** (PLN)
- **Liczba zamówień w roku**

Pytanie: Czy możemy podzielić klientów na segmenty i kierować do nich różne kampanie?

In [ ]:
# Komórka 3 — Generowanie danych klientów
np.random.seed(42)
n_per_segment = 100

klienci = pd.DataFrame({
    'srednia_wartosc': np.concatenate([
        np.random.normal(500, 80, n_per_segment),    # okazjonalni
        np.random.normal(1200, 150, n_per_segment),  # standard
        np.random.normal(2500, 200, n_per_segment)   # premium
    ]),
    'zamowienia_rok': np.concatenate([
        np.random.normal(2, 0.5, n_per_segment),
        np.random.normal(8, 1.5, n_per_segment),
        np.random.normal(20, 3.0, n_per_segment)
    ])
})

# Usuń wartości ujemne
klienci = klienci.clip(lower=0)

print(f"Liczba klientów: {len(klienci)}")
print("\nPodgląd danych:")
klienci.describe().round(1)

In [ ]:
# Komórka 4 — StandardScaler: po co normalizujemy?
print("=== PRZED normalizacją ===")
print(f"srednia_wartosc: zakres {klienci['srednia_wartosc'].min():.0f} - {klienci['srednia_wartosc'].max():.0f} PLN")
print(f"zamowienia_rok:  zakres {klienci['zamowienia_rok'].min():.1f} - {klienci['zamowienia_rok'].max():.1f}")
print("\nBez normalizacji KMeans byłby zdominowany przez 'srednia_wartosc' (większe liczby)!")

scaler = StandardScaler()
X = scaler.fit_transform(klienci[['srednia_wartosc', 'zamowienia_rok']])

print("\n=== PO normalizacji ===")
print(f"Kolumna 0 (wartość): mean={X[:, 0].mean():.3f}, std={X[:, 0].std():.3f}")
print(f"Kolumna 1 (zamówienia): mean={X[:, 1].mean():.3f}, std={X[:, 1].std():.3f}")
print("\nTeraz obie kolumny mają mean≈0, std≈1 — porównywalne!")

In [ ]:
# Komórka 5 — KMeans: trening i przypisanie klastrów
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
kmeans.fit(X)
klienci['segment'] = kmeans.labels_

print("Rozkład klientów per segment:")
print(klienci['segment'].value_counts().sort_index())

print("\nŚrednie cechy per segment:")
print(klienci.groupby('segment')[['srednia_wartosc', 'zamowienia_rok']].mean().round(1))

print(f"\nInertia (suma kwadratów odległości): {kmeans.inertia_:.1f}")

In [ ]:
# Komórka 6 — Wizualizacja klastrów (matplotlib)
fig, ax = plt.subplots(figsize=(10, 6))
kolory = {0: '#636EFA', 1: '#EF553B', 2: '#00CC96'}
nazwy = {0: 'Okazjonalni', 1: 'Standard', 2: 'Premium'}

# Sortowanie segmentów po średniej wartości zamówienia
seg_order = (klienci.groupby('segment')['srednia_wartosc']
             .mean().sort_values().index.tolist())
label_map = {seg: i for i, seg in enumerate(seg_order)}
klienci['segment_label'] = klienci['segment'].map(label_map)
klienci['nazwa_segmentu'] = klienci['segment_label'].map(nazwy)

for seg_label, seg_name in nazwy.items():
    dane = klienci[klienci['segment_label'] == seg_label]
    ax.scatter(dane['srednia_wartosc'], dane['zamowienia_rok'],
               c=kolory[seg_label], label=seg_name, alpha=0.7, s=40)

ax.set_xlabel('Średnia wartość zamówienia [PLN]', fontsize=12)
ax.set_ylabel('Liczba zamówień w roku', fontsize=12)
ax.set_title('Segmentacja klientów — KMeans k=3', fontsize=14)
ax.legend(title='Segment')
plt.tight_layout()
plt.show()

print("\nInterpretacja biznesowa:")
for seg_label, seg_name in nazwy.items():
    dane = klienci[klienci['segment_label'] == seg_label]
    print(f"  {seg_name}: wartość={dane['srednia_wartosc'].mean():.0f} PLN, "
          f"zamówień/rok={dane['zamowienia_rok'].mean():.1f}, "
          f"n={len(dane)}")

---
## CZĘŚĆ 2: scikit-learn — Regresja liniowa

### Scenariusz biznesowy
Masz dane z 200 regionów sprzedaży: wydatki na reklamę TV i radio oraz osiągniętą sprzedaż.  
Pytanie: Jak zwiększyć sprzedaż — inwestować więcej w TV czy w radio?

In [ ]:
# Komórka 7 — Dane sprzedażowe
np.random.seed(42)
n = 200

reklama_tv = np.random.uniform(10, 300, n)
reklama_radio = np.random.uniform(5, 50, n)
szum = np.random.normal(0, 1.5, n)

# Prawdziwa zależność (model to odtworzy z danych)
sprzedaz = 5 + 0.05 * reklama_tv + 0.12 * reklama_radio + szum

df_reklama = pd.DataFrame({
    'tv': reklama_tv,
    'radio': reklama_radio,
    'sprzedaz': sprzedaz
})

print("Dane reklamowe (pierwsze 5 wierszy):")
print(df_reklama.head().round(2))
print(f"\nKorelacja TV-sprzedaż: {df_reklama['tv'].corr(df_reklama['sprzedaz']):.3f}")
print(f"Korelacja Radio-sprzedaż: {df_reklama['radio'].corr(df_reklama['sprzedaz']):.3f}")

In [ ]:
# Komórka 8 — train/test split
X = df_reklama[['tv', 'radio']]
y = df_reklama['sprzedaz']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Zbiór treningowy: {len(X_train)} próbek (80%)")
print(f"Zbiór testowy:    {len(X_test)} próbek (20%)")
print("\nZasada: model NIGDY nie widzi danych testowych podczas treningu!")
print("Zbiór testowy = 'sprawdzian z materiału nieznanego modelu'")

In [ ]:
# Komórka 9 — Trening, predykcja, ocena
model = LinearRegression()
model.fit(X_train, y_train)  # TRENING

y_pred = model.predict(X_test)  # PREDYKCJA

# OCENA
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print("=== Wyniki modelu ===")
print(f"R²   = {r2:.4f}  (1.0 = model perfekcyjny)")
print(f"RMSE = {rmse:.4f} (błąd w jednostkach zmiennej celu)")
print()
print("=== Odkryte współczynniki ===")
print(f"Intercept:         {model.intercept_:.3f}  (prawdziwy: 5.0)")
print(f"Współczynnik TV:   {model.coef_[0]:.4f} (prawdziwy: 0.0500)")
print(f"Współczynnik Radio: {model.coef_[1]:.4f} (prawdziwy: 0.1200)")
print()
print("Interpretacja: każda 1 tys. PLN więcej w radio daje więcej sprzedaży niż TV!")

In [ ]:
# Komórka 10 — Wykres predykcji vs rzeczywistość
fig, axes = plt.subplots(1, 2, figsize=(13, 5))

# Scatter: predykcja vs rzeczywistość
axes[0].scatter(y_test, y_pred, alpha=0.6, color='steelblue', edgecolors='k', lw=0.5)
min_v = min(y_test.min(), y_pred.min())
max_v = max(y_test.max(), y_pred.max())
axes[0].plot([min_v, max_v], [min_v, max_v], 'r--', lw=2, label='Idealna linia')
axes[0].set_xlabel('Sprzedaż rzeczywista [tys. PLN]')
axes[0].set_ylabel('Sprzedaż przewidywana [tys. PLN]')
axes[0].set_title(f'Predykcja vs Rzeczywistość\nR²={r2:.3f}, RMSE={rmse:.3f}')
axes[0].legend()

# Błędy rezydualne
residuals = y_test - y_pred
axes[1].scatter(y_pred, residuals, alpha=0.6, color='coral', edgecolors='k', lw=0.5)
axes[1].axhline(y=0, color='r', linestyle='--', lw=2)
axes[1].set_xlabel('Wartości przewidywane')
axes[1].set_ylabel('Reszty (residuals)')
axes[1].set_title('Analiza reszt')

plt.suptitle('Regresja liniowa — ocena modelu', fontsize=14)
plt.tight_layout()
plt.show()

print("Reszty losowo rozrzucone wokół 0 → model dobrze dopasowany, brak systematycznego błędu")

In [ ]:
# Komórka 11 — Predykcja dla nowych danych
scenariusze = pd.DataFrame({
    'tv': [100, 200, 200, 50],
    'radio': [10, 10, 40, 40]
})

prognozy = model.predict(scenariusze)

print("Prognozy sprzedaży dla różnych scenariuszy reklamowych:")
print(f"{'TV [tys PLN]':>15} {'Radio [tys PLN]':>15} {'Prognoza [tys PLN]':>20}")
print("-" * 55)
for i, (_, row) in enumerate(scenariusze.iterrows()):
    print(f"{row['tv']:>15.0f} {row['radio']:>15.0f} {prognozy[i]:>20.1f}")

print("\nWniosek: zwiększenie budżetu radiowego z 10 do 40 daje więcej sprzedaży niż TV!")
print(f"  Wzrost przez Radio (+30): +{0.12*30:.1f} tys PLN")
print(f"  Wzrost przez TV (+100):   +{0.05*100:.1f} tys PLN")

---
## CZĘŚĆ 3: Plotly — Interaktywne wykresy

**Kluczowa różnica od Matplotlib:**
- Matplotlib → statyczny obraz (PNG/SVG)
- Plotly → interaktywny HTML (hover, zoom, filter)

Spróbuj po uruchomieniu każdego wykresu:
- Najedź myszą na punkt → hover z wartościami
- Kliknij na legendę → ukryj/pokaż serię
- Zaznacz obszar → zoom
- Podwójne kliknięcie → reset widoku

In [ ]:
# Komórka 12 — Scatter plot: segmentacja klientów
fig_scatter = px.scatter(
    klienci,
    x='srednia_wartosc',
    y='zamowienia_rok',
    color='nazwa_segmentu',
    title='Segmentacja klientów — KMeans k=3 (interaktywny Plotly)',
    labels={
        'srednia_wartosc': 'Średnia wartość zamówienia [PLN]',
        'zamowienia_rok': 'Liczba zamówień w roku',
        'nazwa_segmentu': 'Segment'
    },
    color_discrete_sequence=['#636EFA', '#EF553B', '#00CC96'],
    hover_data=['srednia_wartosc', 'zamowienia_rok']
)
fig_scatter.update_traces(marker=dict(size=8, opacity=0.75))
fig_scatter.update_layout(
    legend_title_text='Segment klienta',
    plot_bgcolor='white',
    paper_bgcolor='white'
)
fig_scatter.show()
print("Spróbuj: hover na punkt, kliknij na segment w legendzie, zaznacz obszar (zoom)")

In [ ]:
# Komórka 13 — Scatter z rozmiarem punktu = wartość zamówienia
fig_bubble = px.scatter(
    klienci,
    x='srednia_wartosc',
    y='zamowienia_rok',
    color='nazwa_segmentu',
    size='srednia_wartosc',
    title='Segmentacja klientów — rozmiar punktu = wartość zamówienia',
    labels={
        'srednia_wartosc': 'Średnia wartość zamówienia [PLN]',
        'zamowienia_rok': 'Zamówień w roku',
        'nazwa_segmentu': 'Segment'
    },
    color_discrete_sequence=['#636EFA', '#EF553B', '#00CC96'],
    size_max=20
)
fig_bubble.update_layout(plot_bgcolor='white', paper_bgcolor='white')
fig_bubble.show()
print("Bubble chart: 3 wymiary danych na jednym wykresie — x, y i rozmiar")

In [ ]:
# Komórka 14 — Bar chart: średnie wartości per segment
srednie_seg = (
    klienci.groupby('nazwa_segmentu')[['srednia_wartosc', 'zamowienia_rok']]
    .mean()
    .reset_index()
    .sort_values('srednia_wartosc')
    .round(1)
)

fig_bar = px.bar(
    srednie_seg,
    x='nazwa_segmentu',
    y='srednia_wartosc',
    color='nazwa_segmentu',
    title='Średnia wartość zamówienia per segment klienta',
    labels={
        'srednia_wartosc': 'Średnia wartość zamówienia [PLN]',
        'nazwa_segmentu': 'Segment'
    },
    text_auto='.0f',
    color_discrete_sequence=['#636EFA', '#EF553B', '#00CC96']
)
fig_bar.update_layout(
    showlegend=False,
    plot_bgcolor='white',
    paper_bgcolor='white'
)
fig_bar.update_traces(textposition='outside')
fig_bar.show()

In [ ]:
# Komórka 15 — Line chart: trend sprzedaży 2023 vs 2024
np.random.seed(42)
miesiace = ['Sty', 'Lut', 'Mar', 'Kwi', 'Maj', 'Cze',
            'Lip', 'Sie', 'Wrz', 'Paź', 'Lis', 'Gru']

sprzedaz_2023 = [80 + i*4 + np.random.normal(0, 5) for i in range(1, 13)]
sprzedaz_2024 = [100 + i*6 + np.random.normal(0, 7) for i in range(1, 13)]

df_trend = pd.DataFrame({
    'miesiac': miesiace * 2,
    'sprzedaz': sprzedaz_2023 + sprzedaz_2024,
    'rok': ['2023'] * 12 + ['2024'] * 12
})

fig_line = px.line(
    df_trend,
    x='miesiac',
    y='sprzedaz',
    color='rok',
    title='Trend sprzedaży 2023 vs 2024',
    labels={
        'miesiac': 'Miesiąc',
        'sprzedaz': 'Sprzedaż [tys. PLN]',
        'rok': 'Rok'
    },
    markers=True,
    color_discrete_sequence=['#636EFA', '#EF553B']
)
fig_line.update_layout(
    plot_bgcolor='white',
    paper_bgcolor='white',
    xaxis=dict(showgrid=True, gridcolor='lightgray'),
    yaxis=dict(showgrid=True, gridcolor='lightgray')
)
fig_line.show()
print("Spróbuj: kliknij na '2023' w legendzie żeby porównać tylko 2024")

In [ ]:
# Komórka 16 — Dashboard wielopanelowy
fig_dash = make_subplots(
    rows=2, cols=2,
    subplot_titles=[
        'Segmentacja klientów (scatter)',
        'Wartość per segment (bar)',
        'Trend sprzedaży 2023 vs 2024 (line)',
        'Rozkład wartości zamówień (histogram)'
    ]
)

kolory_seg = {'Okazjonalni': '#636EFA', 'Standard': '#EF553B', 'Premium': '#00CC96'}

# Panel 1,1: Scatter segmentacji
for seg_name, kolor in kolory_seg.items():
    dane_seg = klienci[klienci['nazwa_segmentu'] == seg_name]
    fig_dash.add_trace(
        go.Scatter(
            x=dane_seg['srednia_wartosc'],
            y=dane_seg['zamowienia_rok'],
            mode='markers',
            name=seg_name,
            marker=dict(color=kolor, opacity=0.7, size=6)
        ),
        row=1, col=1
    )

# Panel 1,2: Bar chart
fig_dash.add_trace(
    go.Bar(
        x=srednie_seg['nazwa_segmentu'],
        y=srednie_seg['srednia_wartosc'],
        marker_color=[kolory_seg.get(s, '#636EFA') for s in srednie_seg['nazwa_segmentu']],
        showlegend=False,
        text=srednie_seg['srednia_wartosc'].round(0).astype(int),
        textposition='outside'
    ),
    row=1, col=2
)

# Panel 2,1: Line chart trendu
for rok_val, kolor in [('2023', '#636EFA'), ('2024', '#EF553B')]:
    dane_rok = df_trend[df_trend['rok'] == rok_val]
    fig_dash.add_trace(
        go.Scatter(
            x=dane_rok['miesiac'],
            y=dane_rok['sprzedaz'],
            mode='lines+markers',
            name=f'Sprzedaż {rok_val}',
            marker=dict(color=kolor),
            line=dict(color=kolor),
            showlegend=False
        ),
        row=2, col=1
    )

# Panel 2,2: Histogram
fig_dash.add_trace(
    go.Histogram(
        x=klienci['srednia_wartosc'],
        nbinsx=30,
        marker_color='#AB63FA',
        opacity=0.8,
        showlegend=False,
        name='Rozkład wartości'
    ),
    row=2, col=2
)

fig_dash.update_layout(
    height=750,
    title_text='Dashboard Analityczny — Segmentacja Klientów',
    title_font_size=16,
    plot_bgcolor='white',
    paper_bgcolor='white'
)

# Osie opisów dla każdego panelu
fig_dash.update_xaxes(title_text='Wartość zamówienia [PLN]', row=1, col=1)
fig_dash.update_yaxes(title_text='Zamówień/rok', row=1, col=1)
fig_dash.update_xaxes(title_text='Segment', row=1, col=2)
fig_dash.update_yaxes(title_text='Śr. wartość [PLN]', row=1, col=2)
fig_dash.update_xaxes(title_text='Miesiąc', row=2, col=1)
fig_dash.update_yaxes(title_text='Sprzedaż [tys. PLN]', row=2, col=1)
fig_dash.update_xaxes(title_text='Wartość zamówienia [PLN]', row=2, col=2)
fig_dash.update_yaxes(title_text='Liczba klientów', row=2, col=2)

fig_dash.show()

# Eksport do HTML
import os
output_dir = os.path.dirname(os.path.abspath('advanced_libs_demo.ipynb'))
html_path = os.path.join(output_dir, 'dashboard_klienci.html')
fig_dash.write_html(html_path)
print(f"Dashboard zapisany: {html_path}")
print("Plik HTML możesz otworzyć w przeglądarce lub przesłać do klienta!")

---
## CZĘŚĆ 4: Polars — Brief Mention

Polars to biblioteka do pracy z danymi napisana w Rust.  
**Kluczowa zaleta:** 5-20x szybsza niż Pandas na dużych danych.

| Cecha | Pandas | Polars |
|-------|--------|--------|
| Język implementacji | Python/C | Rust |
| Szybkość (duże dane) | Bazowa | 5-20x szybszy |
| API | Dojrzałe, szeroka dokumentacja | Nowoczesne, rosnące |
| Integracja scikit-learn | Natywna | Przez konwersję |
| Kiedy używać | < 1M wierszy, prototypy | > 1M wierszy, produkcja |

In [ ]:
# Komórka 17 — Benchmark Pandas vs Polars
import time

# Generujemy duży DataFrame
np.random.seed(42)
n_rows = 500_000
df_big = pd.DataFrame({
    'id': range(n_rows),
    'wartosc': np.random.normal(1000, 200, n_rows),
    'kategoria': np.random.choice(['A', 'B', 'C', 'D'], n_rows)
})

# Pandas benchmark
start = time.perf_counter()
wynik_pandas = df_big.groupby('kategoria')['wartosc'].agg(['mean', 'std', 'count'])
czas_pandas = time.perf_counter() - start
print(f"Pandas ({n_rows:,} wierszy): {czas_pandas*1000:.1f} ms")
print(wynik_pandas.round(2))

# Polars benchmark (jeśli zainstalowany)
try:
    import polars as pl
    df_polars = pl.from_pandas(df_big)
    
    start = time.perf_counter()
    wynik_polars = (df_polars
                    .group_by('kategoria')
                    .agg([
                        pl.col('wartosc').mean().alias('mean'),
                        pl.col('wartosc').std().alias('std'),
                        pl.col('wartosc').count().alias('count')
                    ])
                    .sort('kategoria'))
    czas_polars = time.perf_counter() - start
    
    print(f"\nPolars ({n_rows:,} wierszy): {czas_polars*1000:.1f} ms")
    if czas_polars > 0:
        print(f"Polars jest {czas_pandas/czas_polars:.1f}x szybszy niż Pandas")
    print(wynik_polars)
    
except ImportError:
    print("\nPolars nie jest zainstalowany.")
    print("Zainstaluj: uv pip install polars")
    print(f"\nSzacunkowy wynik dla {n_rows:,} wierszy:")
    print(f"  Pandas: ~{czas_pandas*1000:.0f} ms")
    print(f"  Polars: ~{czas_pandas*1000/8:.0f} ms (typowo 5-10x szybszy)")

In [ ]:
# Komórka 18 — Polars: składnia (nawet bez instalacji jako demonstracja)
print("=== Składnia Polars vs Pandas ===")
print()
print("PANDAS:")
print("  df.groupby('kategoria')['wartosc'].mean()")
print("  df[df['wartosc'] > 1000]['id']")
print()
print("POLARS:")
print("  df.group_by('kategoria').agg(pl.col('wartosc').mean())")
print("  df.filter(pl.col('wartosc') > 1000).select('id')")
print()
print("Polars jest bardziej 'wyrażeniowy' — operacje komponujemy metodą łańcuchową.")
print("Instalacja: uv pip install polars")
print("Dokumentacja: https://docs.pola.rs/")

---
## AKTYWNOŚĆ: KMeans + Plotly (10 min)

**Zadanie:** Wygeneruj nowe dane klientów z 4 segmentami i stwórz interaktywny scatter plot.

In [ ]:
# Komórka 19 — AKTYWNOŚĆ: Twój kod tutaj
# TODO: Uzupełnij kod

np.random.seed(123)

# TODO: Wygeneruj dane 200 klientów z 4 segmentami
# Cechy: 'wydatki_miesiac' i 'wiek'
# Segment 0: młodzi, niskie wydatki
# Segment 1: młodzi, wysokie wydatki
# Segment 2: starsi, niskie wydatki
# Segment 3: starsi, wysokie wydatki

klienci_akt = pd.DataFrame({
    'wydatki_miesiac': np.concatenate([
        np.random.normal(300, 50, 50),
        np.random.normal(1500, 200, 50),
        np.random.normal(400, 60, 50),
        np.random.normal(2000, 300, 50)
    ]),
    'wiek': np.concatenate([
        np.random.normal(25, 3, 50),
        np.random.normal(28, 4, 50),
        np.random.normal(55, 6, 50),
        np.random.normal(52, 7, 50)
    ])
})
klienci_akt = klienci_akt.clip(lower=0)

# TODO: Skalowanie i KMeans k=4
scaler_akt = StandardScaler()
X_akt = scaler_akt.fit_transform(klienci_akt[['wydatki_miesiac', 'wiek']])
kmeans_akt = KMeans(n_clusters=4, random_state=42, n_init=10)
kmeans_akt.fit(X_akt)
klienci_akt['segment'] = kmeans_akt.labels_.astype(str)

# TODO: Scatter plot Plotly
fig_akt = px.scatter(
    klienci_akt,
    x='wydatki_miesiac',
    y='wiek',
    color='segment',
    size='wydatki_miesiac',
    title='Segmentacja klientów — 4 grupy (wiek vs wydatki)',
    labels={
        'wydatki_miesiac': 'Wydatki miesięczne [PLN]',
        'wiek': 'Wiek klienta [lata]',
        'segment': 'Segment'
    },
    size_max=15
)
fig_akt.update_layout(plot_bgcolor='white', paper_bgcolor='white')
fig_akt.show()

print("Charakterystyka segmentów:")
print(klienci_akt.groupby('segment')[['wydatki_miesiac', 'wiek']].mean().round(1))

---
## PODSUMOWANIE W13

### Co opanowałeś dziś:

| Temat | Kluczowe narzędzia | Przykład użycia |
|-------|-------------------|------------------|
| **scikit-learn API** | `fit`, `predict`, `score` | Jednolity interfejs dla wszystkich algorytmów |
| **StandardScaler** | `fit_transform`, `transform` | Normalizacja przed KMeans/SVMs |
| **KMeans** | `n_clusters`, `labels_`, `inertia_` | Segmentacja klientów |
| **Train/test split** | `train_test_split(test_size=0.2)` | Uczciwa ocena modelu |
| **Regresja liniowa** | `LinearRegression`, `coef_` | Prognozowanie sprzedaży |
| **Metryki** | `r2_score`, `mean_squared_error` | R² i RMSE = ocena jakości |
| **Plotly Express** | `px.scatter`, `px.bar`, `px.line` | Interaktywne wykresy z 1 linii |
| **Plotly subplots** | `make_subplots`, `go.Scatter` | Dashboard wielopanelowy |
| **Polars** | `pl.DataFrame`, `group_by` | Szybsze przetwarzanie dużych danych |

### Zapowiedź W14: LLM i AI API
- Wywoływanie GPT-4 / Claude przez Python
- Przetwarzanie odpowiedzi JSON
- Prosty chatbot w 30 linijkach kodu
- Przygotowanie: `uv pip install openai` lub `uv pip install anthropic`